Package and their versions:

pip install u8darts[all]==0.34.0

pip install tsfel==0.1.9

In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import TCNModel
from darts.utils.timeseries_generation import linear_timeseries
from darts.utils.likelihood_models import GaussianLikelihood
from darts.utils.data import DualCovariatesSequentialDataset

/Users/canhu/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Quite a few datasets available on darts, and some algorithms use specific datasets

DualCovariatesSequentialDataset

PastCovariatesSequentialDataset

SplitCovariatesShiftedDataset

### TCNModel has some limitation? Not sure why the output_chunk_length can only be 1

In [30]:
# dataset = DualCovariatesSequentialDataset(
#     target_series=target_series,
#     covariates=past_covariates,  
#     input_chunk_length=12,
#     output_chunk_length=1,
# )

# past_dataset = PastCovariatesSequentialDataset(
#     target_series=target_series,
#     covariates=past_covariates,  
#     input_chunk_length=12,
#     output_chunk_length=1,
# )


# shifted_dataset = DualCovariatesShiftedDataset(
#     target_series=target_series,
#     covariates=past_covariates,
#     length =input_chunk_length,
#     shift =1

# )

# from darts.utils.data import SplitCovariatesShiftedDataset

# split_dataset = SplitCovariatesShiftedDataset(
#     target_series=target_series,
#     past_covariates=past_covariates,
#     future_covariates=past_covariates,
#     length=input_chunk_length,
#     shift=output_chunk_length
# )


### Passing windowed dataset into algorithms, instead of letting it perform the windowing under the hood

In [51]:
from darts import TimeSeries
from darts.utils.timeseries_generation import linear_timeseries
from darts.utils.likelihood_models import GaussianLikelihood
from darts.utils.data import PastCovariatesSequentialDataset
import numpy as np

# Generate synthetic target and covariate series
target_series = linear_timeseries(length=100, freq="W").astype(np.float32)
past_covariates_series = linear_timeseries(length=100, freq="W", start_value=10, end_value=50).astype(np.float32)

# Define window parameters
input_chunk_length = 12
output_chunk_length = 4

# Create a dataset for models that support multi-step forecasting
dataset = PastCovariatesSequentialDataset(
    target_series=target_series,
    covariates=past_covariates_series,
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
)


### Able to inspect each window

In [52]:
window_past_target, window_past_covariates, window_static_covariates, window_sample_weights, window_future_target = dataset[0]

In [7]:
from darts.models import NBEATSModel

nbeats = NBEATSModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    n_epochs=10,
    batch_size=16,
    random_state=42,
    likelihood=GaussianLikelihood(),
)

nbeats.fit_from_dataset(dataset, verbose=True)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 6.2 M  | train
-------------------------------------------------------------
6.2 M     Trainable params
1.4 K     Non-trainable params
6.2 M     Total params
24.939    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 6/6 [00:00<00:00,  9.42it/s, train_loss=5.200]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 6/6 [00:00<00:00,  9.41it/s, train_loss=5.200]


NBEATSModel(output_chunk_shift=0, generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=ReLU, input_chunk_length=12, output_chunk_length=4, n_epochs=10, batch_size=16, random_state=42, likelihood=GaussianLikelihood(prior_mu=None, prior_sigma=None, beta_nll=0.0, prior_strength=1.0))

In [75]:
forecast = nbeats.predict(
    n=4,
    series=target_series,               # ✅ Required
    past_covariates=past_covariates_series     # Optional, if you used them during training
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 20.04it/s]


In [78]:
from darts.models import BlockRNNModel

block_rnn = BlockRNNModel(
    model="LSTM",
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    hidden_dim=32,
    n_rnn_layers=2,
    n_epochs=50,
    batch_size=16,
    dropout=0.1,
    random_state=42,
    likelihood=GaussianLikelihood(),
)

block_rnn.fit_from_dataset(dataset, verbose=True)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 13.1 K | train
6 | fc              | Sequential       | 264    | train
-------------------------------------------------------------
13.3 K    Trainable params
0         Non-trainable params
13.3 K    Total params
0.053     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Epoch 0:   0%|          | 0/6 [00:00<?, ?it/s] 

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 52.40it/s, train_loss=-2.44] 

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 50.64it/s, train_loss=-2.44]


BlockRNNModel(output_chunk_shift=0, model=LSTM, hidden_dim=32, n_rnn_layers=2, hidden_fc_sizes=None, dropout=0.1, activation=ReLU, input_chunk_length=12, output_chunk_length=4, n_epochs=50, batch_size=16, random_state=42, likelihood=GaussianLikelihood(prior_mu=None, prior_sigma=None, prior_strength=1.0, beta_nll=0.0))

In [ ]:
forecast = block_rnn.predict(
    n=4,
    series=target_series,               # ✅ Required
    past_covariates=past_covariates_series     # Optional, if you used them during training
)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 72.16it/s]


### Use dataset windows + creating features with tsfel 



In [79]:
window_past_target, window_past_covariates, window_static_covariates, window_sample_weights, window_future_target = dataset[84]

In [77]:
window_past_target

array([[0.02020202],
       [0.03030303],
       [0.04040404],
       [0.05050505],
       [0.06060606],
       [0.07070707],
       [0.08080808],
       [0.09090909],
       [0.1010101 ],
       [0.11111111],
       [0.12121212],
       [0.13131313]], dtype=float32)

In [72]:
from darts.utils.data import PastCovariatesSequentialDataset
import tsfel
import pandas as pd
from darts import TimeSeries


# 1. Setup dataset
dataset = PastCovariatesSequentialDataset(
    target_series=target_series,
    covariates=past_covariates_series,
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
)

# 2. TSFEL config
cfg = tsfel.get_features_by_domain()

# 3. Extract features from each window in the dataset
features_list = []

for i, (past_target, _, _, _, tar) in enumerate(dataset):

    window_start = target_series.time_index[i]
    window_end = target_series.time_index[i + input_chunk_length - 1]
    time_index = target_series.time_index[i : i + input_chunk_length]

    past_target_ts = TimeSeries.from_times_and_values(
        times=time_index,
        values=past_target
    )

    df_window = past_target_ts.pd_dataframe().reset_index().rename(columns={"0": "signal"}) # dataframe with time and signal columns

    # Extract TSFEL features
    features = tsfel.time_series_features_extractor(cfg, df_window[['signal']], verbose=0) # dataframe of generated features (1, 132)
    
    # Tag with timestamp from end of input window
    features["index"] = df_window['time']
    features_list.append(features)

# 4. Assemble into a covariates TimeSeries
features_df = pd.concat(features_list).set_index("index").sort_index()
tsfel_covariates = TimeSeries.from_dataframe(features_df)

# 5. (Optional) Align with the target series if needed
common_index = target_series.time_index.intersection(tsfel_covariates.time_index) # 85
target_trimmed = target_series.drop_before(common_index[0]).drop_after(common_index[-1]).astype(np.float32)  # 83
covariates_trimmed = tsfel_covariates.drop_before(common_index[0]).drop_after(common_index[-1]).astype(np.float32) # 83


/var/folders/kt/yzrbp9yd45q1_4dbfnz81sqm0000gn/T/ipykernel_30676/3084242449.py:35: UserWarning: Using default sampling frequency set in configuration file.
  features = tsfel.time_series_features_extractor(cfg, df_window[['signal']], verbose=0) # dataframe of generated features (1, 132)
/var/folders/kt/yzrbp9yd45q1_4dbfnz81sqm0000gn/T/ipykernel_30676/3084242449.py:35: UserWarning: Using default sampling frequency set in configuration file.
  features = tsfel.time_series_features_extractor(cfg, df_window[['signal']], verbose=0) # dataframe of generated features (1, 132)
/var/folders/kt/yzrbp9yd45q1_4dbfnz81sqm0000gn/T/ipykernel_30676/3084242449.py:35: UserWarning: Using default sampling frequency set in configuration file.
  features = tsfel.time_series_features_extractor(cfg, df_window[['signal']], verbose=0) # dataframe of generated features (1, 132)
/var/folders/kt/yzrbp9yd45q1_4dbfnz81sqm0000gn/T/ipykernel_30676/3084242449.py:35: UserWarning: Using default sampling frequency set in 

Workflow:
1. Create slicing/windowing via PastCovariatesSequentialDataset (85 windows of 12+4 from dataset of length 100: 100-12-4+1=85)
2. Generate one set of TSFEL features for each window (132 features)

For each window:
- Target series: 12 + 4 time stamps
- TSFEL covariate: 132 features -- without time stamp

Idea:
- Compute global features with rolling window -- to make them time-reliant 
- Treat them as covariate time series (length 85 or 83)
- The final training dataset is (83-12-4+1=68)

In [ ]:
# len(features_list) # 85
# features_list[0].shape  # (1, 132)

# features_df.shape   # (85, 131)
# tsfel_covariates.shape # (85, 131, 1)

# target_series.time_index[0] # 2000-01-02 00:00:00
# tsfel_covariates.time_index[0] # 2000-01-02 00:00:00

window_past_target, window_past_covariates, _, _, window_future_target = dataset[80]
window_past_target.shape # (12,1)
tsfel_covariates[0].shape # (1, 131, 1)


(1, 132)

In [ ]:
tsfel_dataset = PastCovariatesSequentialDataset(
    target_series=target_trimmed,            # aligned trimmed target
    covariates=covariates_trimmed,           # aligned TSFEL features
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
)


In [ ]:
print(len(target_series))
print(len(target_trimmed))
print(len(tsfel_dataset))

window_past_target, window_past_covariates, _, _, window_future_target = tsfel_dataset[0]
# window_past_target.shape     # (12,1)
# window_past_covariates.shape # (12, 131)
# window_future_target.shape   # (4,1)

100
83
68


In [75]:
from darts.models import NBEATSModel

nbeats = NBEATSModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    n_epochs=10,
    batch_size=16,
    random_state=42,
    likelihood=GaussianLikelihood(),
)

# 5. Train the model directly from the dataset
nbeats.fit_from_dataset(tsfel_dataset, verbose=True)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 18.6 M | train
-------------------------------------------------------------
18.6 M    Trainable params
10.8 K    Non-trainable params
18.6 M    Total params
74.360    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|██████████| 5/5 [00:00<00:00,  9.26it/s, train_loss=6.640]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 5/5 [00:00<00:00,  9.25it/s, train_loss=6.640]


NBEATSModel(output_chunk_shift=0, generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=ReLU, input_chunk_length=12, output_chunk_length=4, n_epochs=10, batch_size=16, random_state=42, likelihood=GaussianLikelihood(prior_mu=None, prior_sigma=None, prior_strength=1.0, beta_nll=0.0))